In [3]:
import warnings
import numpy as np
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Input, DepthwiseConv2D, Activation, BatchNormalization, Conv2D, GlobalAveragePooling2D, GlobalMaxPooling2D, Reshape, Dropout
from keras.applications.imagenet_utils import decode_predictions
from keras import backend as K

In [44]:
def  _conv_block (inputs, filters_num, kernel = (3,3), strides = (1,1)):
    x = Conv2D(filters_num, kernel, padding = 'same', 
               use_bias = False, strides = strides, name = 'conv1')(inputs)
    x = BatchNormalization(name = 'conv1_bn')(x)
    x = Activation(relu6, name = 'conv1_relu')(x)
    return x


def _depthwise_conv_block (inputs, pointwise_conv_filters_num, depth_multiplier = 1, strides = (1,1), block_id = 1):
    
    
    '''
        keras.layers.DepthwiseConv2D(kernel_size, strides=(1, 1), 
                                 padding='valid', depth_multiplier=1, 
                                 data_format=None, dilation_rate=(1, 1), 
                                 activation=None, use_bias=True, 
                                 depthwise_initializer='glorot_uniform', 
                                 bias_initializer='zeros', 
                                 depthwise_regularizer=None, 
                                 bias_regularizer=None, activity_regularizer=None, 
                                 depthwise_constraint=None, bias_constraint=None)
    '''
    x = DepthwiseConv2D((3,3), strides = strides, depth_multiplier = depth_multiplier, padding = 'same',
                        use_bias=False, name = 'conv_dw_%d' % block_id)(inputs)
    x = BatchNormalization(name = 'conv_dw_%d_bn' % block_id)(x)
    x = Activation(relu6, name = 'conv_dw_%d_relu' % block_id)(x)
    
    x = Conv2D(pointwise_conv_filters_num, (1,1), strides = (1,1), padding = 'same',use_bias=False,
                name = 'conv_pw_%d' % block_id)(x)
    x = BatchNormalization(name = 'conv_pw_%d_bn' % block_id)(x)
    x = Activation(relu6, name = 'conv_pw_%d_relu' % block_id)(x)
    
    return x

def MobileNet(input_shape, classes, dropout, depth_multiplier = 1):
    
    img_input = Input(shape=input_shape)

    # 224,224,3 -> 112,112,32
    x = _conv_block(img_input, 32, strides=(2, 2))
    print(x.shape)

    # 112,112,32 -> 112,112,64
    x = _depthwise_conv_block(x, 64, depth_multiplier, block_id=1)
    print(x.shape)
    # 112,112,64 -> 56,56,128
    x = _depthwise_conv_block(x, 128, depth_multiplier,
                              strides=(2, 2), block_id=2)
    print(x.shape)
    # 56,56,128 -> 56,56,128
    x = _depthwise_conv_block(x, 128, depth_multiplier, block_id=3)
    print(x.shape)

    # 56,56,128 -> 28,28,256
    x = _depthwise_conv_block(x, 256, depth_multiplier,
                              strides=(2, 2), block_id=4)
    print(x.shape)
    
    # 28,28,256 -> 28,28,256
    x = _depthwise_conv_block(x, 256, depth_multiplier, block_id=5)
    print(x.shape)

    # 28,28,256 -> 14,14,512
    x = _depthwise_conv_block(x, 512, depth_multiplier,
                              strides=(2, 2), block_id=6)
    print(x.shape)
    
    # 14,14,512 -> 14,14,512
    x = _depthwise_conv_block(x, 512, depth_multiplier, block_id=7)
    x = _depthwise_conv_block(x, 512, depth_multiplier, block_id=8)
    x = _depthwise_conv_block(x, 512, depth_multiplier, block_id=9)
    x = _depthwise_conv_block(x, 512, depth_multiplier, block_id=10)
    x = _depthwise_conv_block(x, 512, depth_multiplier, block_id=11)
    print(x.shape)

    # 14,14,512 -> 7,7,1024
    x = _depthwise_conv_block(x, 1024, depth_multiplier,
                              strides=(2, 2), block_id=12)
    print(x.shape)
    x = _depthwise_conv_block(x, 1024, depth_multiplier, block_id=13)
    print(x.shape)

    # 7,7,1024 -> 1,1,1024
    x = GlobalAveragePooling2D()(x)
    print(x.shape)
    x = Reshape((1, 1, 1024), name='reshape_1')(x)
    print(x.shape)
    x = Dropout(dropout, name='dropout')(x)
    print(x.shape)
    x = Conv2D(classes, (1, 1),padding='same', name='conv_preds')(x)
    print(x.shape)
    x = Activation('softmax', name='act_softmax')(x)
    print(x.shape)
    x = Reshape((classes,), name='reshape_2')(x)
    print(x.shape)

    inputs = img_input

    model = Model(inputs, x, name='mobilenet_1_0_224_tf')
    model_name = 'mobilenet_1_0_224_tf.h5'
    model.load_weights(model_name)

    return model


In [46]:
def relu6(x):
    return K.relu(x, max_value = 6)
    
def preprocess_input(x):
    x /= 255.
    x -= 0.5
    x *= 2
    return x

if __name__ =='__main__':

    model = MobileNet(input_shape = (224,224,3),classes = 1000, dropout = 1e-3)
    
    #读取图片信息， 图片地址，load图片，nparray
    img_path = 'elephant.jpg'
    img = image.load_img(img_path, target_size = (224,224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    x = preprocess_input(x)
    print('input image shape', x.shape)
    
    preds = model.predict(x)
    print(np.argmax(preds))
    print ('Predicted', decode_predictions(preds,1))
    

(?, 112, 112, 32)
(?, 112, 112, 64)
(?, 56, 56, 128)
(?, 56, 56, 128)
(?, 28, 28, 256)
(?, 28, 28, 256)
(?, 14, 14, 512)
(?, 14, 14, 512)
(?, 7, 7, 1024)
(?, 7, 7, 1024)
(?, 1024)
(?, 1, 1, 1024)
(?, 1, 1, 1024)
(?, 1, 1, 1000)
(?, 1, 1, 1000)
(?, 1000)
input image shape (1, 224, 224, 3)
386
40960/35363 [==================================] - 0s 4us/step
Predicted [[('n02504458', 'African_elephant', 0.7636115)]]
